In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import gc

gc.collect()

31

In [3]:
from huggingface_hub import login
import os

token_file_path = 'secrets.txt'

if os.path.exists(token_file_path):
    try:
        with open(token_file_path, 'r') as f:
            hf_token = f.read().strip()  # .strip() removes any leading/trailing whitespace

        login(token=hf_token)
        print("Successfully logged in to Hugging Face!")

    except Exception as e:
        print(f"An error occurred while trying to read the token file or log in: {e}")
else:
    print(f"Token file not found at {token_file_path}. Please create the file and add your token.")

/home/jovyan/.local/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Successfully logged in to Hugging Face!


In [4]:
from src.data_utils import SIFT50MDataset
import os
from datasets import load_dataset, Dataset

sift_dataset = load_dataset(
    'amazon-agi/SIFT-50M',
    #name='closed_ended_comparison',
    name = 'closed_ended_content_level',
    split='train',
    trust_remote_code=True
)

sift_dataset_eval = load_dataset(
    'amazon-agi/SIFT-50M',
    #name='closed_ended_comparison',
    name = 'closed_ended_content_level',
    split='validation',
    trust_remote_code=True
) 

# Define the allowed data sources
allowed_values = ["common_voice_de", "common_voice_en", "vctk_en"]

# Define a filter function
def filter_data_source(example):
    return example["data_source"] in allowed_values

filtered_sift_data = sift_dataset.filter(filter_data_source)
filtered_sift_data_eval = sift_dataset_eval.filter(filter_data_source)

#df = sift_dataset.select(range(1000)).to_pandas()
print(len(filtered_sift_data))
print(len(filtered_sift_data_eval))
#filtered_df = df[df["data_source"].isin(allowed_values)]

# Count number of entries per value
#counts = filtered_df["data_source"].value_counts()
#print(counts)
#sift_data = Dataset.from_pandas(filtered_df)
# Define the base datasets paths (replace with your actual paths)

print('Training set')
counts = filtered_sift_data.to_pandas()["data_source"].value_counts()
print(counts)
print('Evaluation set')
counts = filtered_sift_data_eval.to_pandas()["data_source"].value_counts()


Filter: 100%|██████████| 77988/77988 [00:01<00:00, 45356.48 examples/s]


3966476
25554
Training set
data_source
common_voice_en    2570993
common_voice_de    1227053
vctk_en             168430
Name: count, dtype: int64
Evaluation set


In [5]:
print(counts)

data_source
common_voice_en    15293
common_voice_de    10261
Name: count, dtype: int64


In [6]:
del counts
gc.collect()

0

In [16]:
filtered_df.iloc[4].messages

array([{'role': 'user', 'content': array([{'text': None, 'audio_path': '/path/to/common_voice_de_27994683.wav'},
              {'text': 'Wie viele Endrundenteilnehmer wurden in jeder Gruppe eingeteilt?', 'audio_path': None}],
             dtype=object)}                                                                                     ,
       {'role': 'assistant', 'content': array([{'text': 'Vier Mannschaften wurden in jeder Gruppe eingeteilt.', 'audio_path': None}],
             dtype=object)}                                                                                                          ],
      dtype=object)

In [1]:
import pandas as pd

csv_de = pd.read_csv('./data/common_voice_en_mapping.csv')

In [2]:
csv_de['audio_path'].unique

<bound method Series.unique of 0         /home/jovyan/.cache/huggingface/datasets/downl...
1         /home/jovyan/.cache/huggingface/datasets/downl...
2         /home/jovyan/.cache/huggingface/datasets/downl...
3         /home/jovyan/.cache/huggingface/datasets/downl...
4         /home/jovyan/.cache/huggingface/datasets/downl...
                                ...                        
936330    /home/jovyan/.cache/huggingface/datasets/downl...
936331    /home/jovyan/.cache/huggingface/datasets/downl...
936332    /home/jovyan/.cache/huggingface/datasets/downl...
936333    /home/jovyan/.cache/huggingface/datasets/downl...
936334    /home/jovyan/.cache/huggingface/datasets/downl...
Name: audio_path, Length: 936335, dtype: object>

In [15]:
dataset = load_dataset("mozilla-foundation/common_voice_15_0", 'en', split="train", trust_remote_code=True)
print(len(dataset))

1070066


In [ ]:
filtered_df_1 = filtered_df[filtered_df["data_source"].isin(['vctk_en'])]
print(filtered_df_1.iloc[1]['messages']) # filterining for a specific dataset

In [4]:
total_len = len(sift_data)
eval_len = int(0.2*total_len)

eval_ds = sift_data.select(range(eval_len))
train_ds = sift_data.select(range(eval_len,total_len))


In [ ]:
base_datasets_root = "/home/jovyan/.cache/huggingface/datasets"
base_datasets_paths = {
    "common_voice_de": None, # No longer needs a path, handled by load_dataset
    #"multilingual_librispeech_de": None, # No longer needs a path, handled by load_dataset
    "common_voice_en": None, # No longer needs a path, handled by load_dataset
    "vctk_en": "./vctk_corpus" # VCTK still needs a root path for torchaudio
}

In [5]:
%reload_ext autoreload
    
from src.train_qlora import train_model
from transformers import AutoProcessor
from src.data_utils import SIFT50MDataset
from src.data_collator import CustomDataCollator


def main():
    processor = AutoProcessor.from_pretrained("Qwen/Qwen2-Audio-7B-Instruct")
    data_collator = CustomDataCollator(processor)

    sift_iterable_dataset_eval = SIFT50MDataset(sift_dataset=filtered_sift_data_eval, base_datasets_paths=base_datasets_paths)
    sift_iterable_dataset_train = SIFT50MDataset(sift_dataset=filtered_sift_data, base_datasets_paths=base_datasets_paths)

    train_model( eval_ds = sift_iterable_dataset_eval,train_ds = sift_iterable_dataset_train, processor=processor, custom_data_collator=data_collator)

In [6]:
main()

Loading checkpoint shards: 100%|██████████| 5/5 [01:32<00:00, 18.56s/it]


['common_voice_en_23843789']


The tokenizer has new PAD/BOS/EOS tokens that differ from the model config and generation config. The model config and generation config were aligned accordingly, being updated with the tokenizer's values. Updated tokens: {'eos_token_id': 151645, 'pad_token_id': 151643}.


['common_voice_en_23843789']
['common_voice_en_33932746']


It is strongly recommended to pass the `sampling_rate` argument to `WhisperFeatureExtractor()`. Failing to do so can result in silent errors that might be hard to debug.
It is strongly recommended to pass the `sampling_rate` argument to `WhisperFeatureExtractor()`. Failing to do so can result in silent errors that might be hard to debug.


...

It is strongly recommended to pass the `sampling_rate` argument to `WhisperFeatureExtractor()`. Failing to do so can result in silent errors that might be hard to debug.
It is strongly recommended to pass the `sampling_rate` argument to `WhisperFeatureExtractor()`. Failing to do so can result in silent errors that might be hard to debug.


['common_voice_en_21522166']
['common_voice_en_26965157']
['common_voice_en_25805935']
['common_voice_en_24843873']
File not found: /path/to/common_voice_en_24843873.wav. Skipping this file.
No valid audio files found for a sample. Skipping.


It is strongly recommended to pass the `sampling_rate` argument to `WhisperFeatureExtractor()`. Failing to do so can result in silent errors that might be hard to debug.
It is strongly recommended to pass the `sampling_rate` argument to `WhisperFeatureExtractor()`. Failing to do so can result in silent errors that might be hard to debug.


['common_voice_en_23786487']
['common_voice_de_25588250']
['p323_313']
['common_voice_en_18598537']


KeyboardInterrupt: 